In [ ]:
"H2"

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B
from qiskit.quantum_info import Z2Symmetries  # For additional qubit reduction
from pyscf import gto, scf, fci

In [2]:
# Improved with better basis set
basis = 'cc-pVDZ'  # Better basis set for H₂
charge = 0
spin = 0
dists = np.linspace(0.4, 4.0, 14)

vqe_energies, uhf_energies, fci_energies = [], [], []

In [ ]:




for d in dists:
    coords = [[0, 0, -d/2], [0, 0, d/2]]
    atom_spec = f"H 0 0 {-d/2}; H 0 0 {d/2}"
    
    # Classical reference calculations
    mol = gto.Mole()
    mol.atom = atom_spec
    mol.basis = basis
    mol.charge = charge
    mol.spin = spin
    mol.build()
    
    mf = scf.RHF(mol).run()
    uhf_energy = mf.e_tot
    
    cisolver = fci.FCI(mol, mf.mo_coeff)
    fci_energy, _ = cisolver.kernel()
    
    # VQE with improved configuration
    driver = PySCFDriver(atom=atom_spec, unit=DistanceUnit.ANGSTROM,
                       charge=charge, spin=spin, basis=basis)
    problem = driver.run()
    
    # Enhanced mapper with two-qubit reduction
    mapper = ParityMapper(num_particles=problem.num_particles)
    hamiltonian = problem.hamiltonian.second_q_op()
    qubit_op = mapper.map(hamiltonian)
    
    # Try to apply Z2 symmetry reduction (if available)
    try:
        # Additional qubit reduction for H₂
        z2_symmetries = Z2Symmetries.find_Z2_symmetries(qubit_op)
        qubit_op = z2_symmetries.taper(qubit_op)
        print(f"Reduced qubits from {mapper.num_qubits} to {qubit_op.num_qubits}")
    except:
        print("Using standard mapping without additional reduction")
    
    # Improved initial point (small random values)
    init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, problem.num_particles, mapper, initial_state=init_state)
    
    # Better initialization
    init_point = np.random.random(ansatz.num_parameters) * 0.1
    
    # More suitable optimizer for H₂
    optimizer = L_BFGS_B(maxiter=100)
    
    # Use primitive with more shots for better statistics
    estimator = Estimator(options={"shots": 8192})
    
    # Run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=init_point)
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    # Get interpreted energy
    if hasattr(problem, 'interpret'):
        vqe_energy = problem.interpret(result).total_energies[0].real
    else:
        vqe_energy = result.eigenvalue.real + problem.energy_shift
    
    vqe_energies.append(vqe_energy)
    uhf_energies.append(uhf_energy)
    fci_energies.append(fci_energy)
    print(f"d={d:.2f} | VQE={vqe_energy:.6f} | UHF={uhf_energy:.6f} | FCI={fci_energy:.6f}")

# Plotting with improved styling
plt.figure(figsize=(8, 5))
plt.plot(dists, vqe_energies, 'o-', label='VQE', color='limegreen', linewidth=2)
plt.plot(dists, uhf_energies, 's-', label='RHF', color='red', linewidth=2)
plt.plot(dists, fci_energies, 'x--', label='FCI (Exact)', color='black', linewidth=2)
plt.xlabel("H-H Distance (Å)", fontsize=12)
plt.ylabel("Energy (Hartree)", fontsize=12)
plt.title(f"H$_2$ Potential Energy Curve ({basis} basis)", fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Calculate dissociation energy
min_index = np.nanargmin(vqe_energies)
min_distance = dists[min_index]
min_energy = vqe_energies[min_index]
dissoc_energy = min_energy - vqe_energies[-1]

print("\n=== H₂ Ground State Analysis ===")
print(f"Minimum energy: {min_energy:.6f} Hartree")
print(f"At bond distance: {min_distance:.2f} Å")
print(f"Dissociation energy: {-dissoc_energy:.6f} Hartree ({-dissoc_energy*627.5:.2f} kcal/mol)")
print(f"Error vs FCI: {(min_energy-fci_energies[min_index])*1000:.3f} mHartree")

In [ ]:
"Ne"

In [4]:
import numpy as np
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
# Fix these imports to use qiskit_algorithms instead of qiskit.algorithms
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit_aer.primitives import Estimator as AerEstimator
from pyscf import gto, scf, mcscf

In [ ]:


ATOM_STR = "Ne 0.0 0.0 0.0"
BASIS = "cc-pVDZ"  # Better basis for neon

# --- PySCF Molecule ---
def get_pyscf_mol():
    mol = gto.Mole()
    mol.atom = ATOM_STR
    mol.basis = BASIS
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

# --- Qiskit Nature Problem ---
def get_qiskit_problem():
    driver = PySCFDriver(
        atom=ATOM_STR,
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
        basis=BASIS
    )
    return driver.run()

# --- VQE with GPU Acceleration ---
def run_vqe(problem):
    # 1. Apply Active Space Transformation
    active_space_transformer = ActiveSpaceTransformer(
        num_electrons=(4, 4),  # 8 valence electrons (4 alpha, 4 beta)
        num_spatial_orbitals=5,  # 5 spatial orbitals (2s, 2p)
        # Skip the 1s core orbital (index 0)
        active_orbitals=[1, 2, 3, 4, 5]
    )
    active_problem = active_space_transformer.transform(problem)
    print(f"Active space: {sum(active_problem.num_particles)} electrons in {active_problem.num_spatial_orbitals} orbitals")
    
    # 2. Use Bravyi-Kitaev mapping for better locality properties
    mapper = BravyiKitaevMapper()
    
    # 3. Create ansatz with appropriate initial state
    init_state = HartreeFock(active_problem.num_spatial_orbitals, active_problem.num_particles, mapper)
    ansatz = UCCSD(active_problem.num_spatial_orbitals, active_problem.num_particles, mapper, initial_state=init_state)
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # 4. Better initial point (small random values)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # 5. Use GPU-accelerated estimator
    try:
        estimator = AerEstimator(
            run_options={"shots": 8192},
            approximation=True,
            device="GPU"
        )
        print("Using GPU acceleration")
    except Exception:
        print("GPU not available, falling back to CPU")
        estimator = AerEstimator(run_options={"shots": 8192})
    
    # 6. Better optimizer for complex parameter space
    optimizer = L_BFGS_B(maxiter=500)
    
    # 7. Run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(active_problem.hamiltonian.second_q_op())
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    # 8. Return interpreted energy including the energy shift from frozen core
    return active_problem.interpret(result).total_energies[0].real

# --- Reference CASSCF calculation ---
def run_casscf(mol):
    # Run mean-field first
    myhf = scf.RHF(mol)
    myhf.kernel()
    
    # Run CASSCF with same active space as VQE (8e, 5o)
    # Corresponds to valence space for neon
    mycas = mcscf.CASSCF(myhf, 5, 8)
    casscf_energy = mycas.kernel()[0]
    return casscf_energy

# --- Main ---
try:
    problem = get_qiskit_problem()
    print(f"Basis: {BASIS}")
    print(f"Full system - Spatial orbitals: {problem.num_spatial_orbitals}, Particles: {problem.num_particles}")

    vqe_energy = run_vqe(problem)

    mol = get_pyscf_mol()
    hf_energy = scf.RHF(mol).kernel()
    casscf_energy = run_casscf(mol)

    print("\n=== Neon Ground State Energy (Hartree) ===")
    print(f"VQE: {vqe_energy:.6f}")
    print(f"HF:  {hf_energy:.6f}")
    print(f"CASSCF(8e,5o): {casscf_energy:.6f}")
    print(f"Correlation energy: {vqe_energy - hf_energy:.6f}")

except Exception as e:
    print("Calculation failed:", str(e))
    import traceback
    traceback.print_exc()

In [ ]:
"H2O"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP
from qiskit_aer.primitives import Estimator as AerEstimator
from pyscf import gto, scf, mcscf

# Water molecule specification (equilibrium geometry)
ATOM_STR = "O 0.0 0.0 0.0; H 0.0 0.757 0.587; H 0.0 -0.757 0.587"
BASIS = "cc-pVDZ"  # Good balance for water

# --- PySCF Molecule ---
def get_pyscf_mol():
    mol = gto.Mole()
    mol.atom = ATOM_STR
    mol.basis = BASIS
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

# --- Qiskit Nature Problem ---
def get_qiskit_problem():
    driver = PySCFDriver(
        atom=ATOM_STR,
        unit=DistanceUnit.ANGSTROM,
        charge=0,
        spin=0,
        basis=BASIS
    )
    return driver.run()

# --- VQE with GPU Acceleration ---
def run_vqe(problem):
    # 1. Apply Active Space Transformation - focus on valence electrons
    active_space_transformer = ActiveSpaceTransformer(
        num_electrons=(4, 4),  # 8 valence electrons (4 alpha, 4 beta)
        num_spatial_orbitals=7,  # 7 spatial orbitals 
        # Skip the oxygen 1s orbital (index 0)
        active_orbitals=[1, 2, 3, 4, 5, 6, 7]
    )
    active_problem = active_space_transformer.transform(problem)
    print(f"Active space: {sum(active_problem.num_particles)} electrons in {active_problem.num_spatial_orbitals} orbitals")
    
    # 2. Use Parity mapping with Z2 symmetry reduction
    mapper = ParityMapper(num_particles=active_problem.num_particles)
    
    # 3. Create ansatz with appropriate initial state
    init_state = HartreeFock(active_problem.num_spatial_orbitals, active_problem.num_particles, mapper)
    ansatz = UCCSD(active_problem.num_spatial_orbitals, active_problem.num_particles, mapper, initial_state=init_state)
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # 4. Better initial point (small random values with seed for reproducibility)
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # 5. Use GPU-accelerated estimator
    try:
        estimator = AerEstimator(
            run_options={"shots": 8192},
            approximation=True,
            device="GPU"
        )
        print("Using GPU acceleration")
    except Exception:
        print("GPU not available, falling back to CPU")
        estimator = AerEstimator(run_options={"shots": 8192})
    
    # 6. SLSQP optimizer with appropriate iterations
    optimizer = SLSQP(maxiter=400)
    
    # 7. Run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(active_problem.hamiltonian.second_q_op())
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    
    # 8. Return interpreted energy including the energy shift from frozen core
    return active_problem.interpret(result).total_energies[0].real

# --- Reference CASSCF calculation ---
def run_casscf(mol):
    # Run mean-field first
    myhf = scf.RHF(mol)
    myhf.kernel()
    
    # Run CASSCF with same active space as VQE (8e, 7o)
    mycas = mcscf.CASSCF(myhf, 7, 8)
    casscf_energy = mycas.kernel()[0]
    return casscf_energy, myhf.e_tot

# --- Calculate dipole moment ---
def calculate_dipole(mol):
    myhf = scf.RHF(mol)
    myhf.kernel()
    return myhf.dip_moment()

# --- Main ---
try:
    # Quantum calculation
    problem = get_qiskit_problem()
    print(f"Basis: {BASIS}")
    print(f"Full system - Spatial orbitals: {problem.num_spatial_orbitals}, Particles: {problem.num_particles}")

    vqe_energy = run_vqe(problem)

    # Classical reference calculations
    mol = get_pyscf_mol()
    casscf_energy, hf_energy = run_casscf(mol)
    dipole = calculate_dipole(mol)
    
    # Energy results
    print("\n=== Water (H₂O) Ground State Properties ===")
    print(f"VQE Energy: {vqe_energy:.6f} Hartree")
    print(f"HF Energy:  {hf_energy:.6f} Hartree")
    print(f"CASSCF(8e,7o) Energy: {casscf_energy:.6f} Hartree")
    print(f"Correlation energy: {vqe_energy - hf_energy:.6f} Hartree")
    
    # Dipole moment
    print(f"\nDipole moment: {np.linalg.norm(dipole):.4f} Debye")
    print(f"Dipole components (x,y,z): {dipole}")

except Exception as e:
    print("Calculation failed:", str(e))
    import traceback
    traceback.print_exc()

In [6]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA, L_BFGS_B, SLSQP

from pyscf import gto, scf, fci, mcscf

In [ ]:



class MoleculeVQE:
    """Perform VQE simulations on various molecules with best practices for each."""
    
    def __init__(self, molecule_type="h2", verbose=True):
        """Initialize simulation with molecule type."""
        self.molecule_type = molecule_type.lower()
        self.verbose = verbose
        self.results = {}
        
        # Configure based on molecule type
        self._configure_molecule()
        
    def _configure_molecule(self):
        """Configure settings based on molecule type."""
        if self.molecule_type == "h2":
            self.basis = "cc-pVDZ"  # Better basis for H₂
            self.charge = 0
            self.spin = 0
            self.active_space = None  # No active space needed for H₂
            self.mapper_type = "parity"
            self.optimizer_name = "COBYLA"
            self.optimizer_kwargs = {"maxiter": 200}
            self.geometry_func = self._get_h2_geometry
            self.scan_range = (0.4, 3.0, 14)  # Start, end, points
            self.scan_label = "H-H Distance (Å)"
            self.title = "H₂ Potential Energy Curve"
            
        elif self.molecule_type == "neon" or self.molecule_type == "ne":
            self.basis = "cc-pVDZ"  # Better basis for Ne
            self.charge = 0
            self.spin = 0
            self.active_space = {
                "num_electrons": (4, 4),  # 8 valence electrons
                "num_spatial_orbitals": 5,  # 5 orbitals
                "active_orbitals": [1, 2, 3, 4, 5]  # Skip 1s core orbital
            }
            self.mapper_type = "bravyi-kitaev"
            self.optimizer_name = "L_BFGS_B"
            self.optimizer_kwargs = {"maxiter": 500}
            self.geometry_func = self._get_ne_geometry
            self.scan_range = (0.0, 0.0, 1)  # Single point for atom
            self.scan_label = "Ne (Å)"
            self.title = "Ne Ground State Energy"
            
        elif self.molecule_type == "h2o" or self.molecule_type == "water":
            self.basis = "cc-pVDZ"  # Better basis for H₂O
            self.charge = 0
            self.spin = 0
            self.active_space = {
                "num_electrons": (4, 4),  # 8 valence electrons
                "num_spatial_orbitals": 7,  # 7 orbitals
                "active_orbitals": [1, 2, 3, 4, 5, 6, 7]  # Skip 1s core orbital
            }
            self.mapper_type = "parity"
            self.optimizer_name = "SLSQP"
            self.optimizer_kwargs = {"maxiter": 400}
            self.geometry_func = self._get_h2o_geometry
            self.scan_range = (0.8, 1.8, 8)  # Start, end, points
            self.scan_label = "O-H Bond Length (Å)"
            self.title = "H₂O Potential Energy Curve"
            
        else:
            raise ValueError(f"Unsupported molecule type: {self.molecule_type}")
            
        # Print configuration
        if self.verbose:
            self._print_config()
            
    def _print_config(self):
        """Print configuration details."""
        print(f"\n{'=' * 50}")
        print(f"Configuration for {self.molecule_type.upper()}")
        print(f"{'=' * 50}")
        print(f"Basis set: {self.basis}")
        print(f"Charge: {self.charge}, Spin: {self.spin}")
        print(f"Active space: {self.active_space}")
        print(f"Mapper: {self.mapper_type}")
        print(f"Optimizer: {self.optimizer_name} with {self.optimizer_kwargs}")
        print(f"{'=' * 50}\n")
        
    def _get_h2_geometry(self, distance):
        """Get H₂ geometry with variable bond length."""
        return [
            ("H", (0.0, 0.0, 0.0)),
            ("H", (0.0, 0.0, distance))
        ]
        
    def _get_ne_geometry(self, _):
        """Get Ne atom geometry (doesn't depend on distance)."""
        return [
            ("Ne", (0.0, 0.0, 0.0))
        ]
        
    def _get_h2o_geometry(self, bond_length):
        """Get H₂O geometry with variable O-H bond length."""
        angle = 104.5 * np.pi / 180  # ~104.5 degrees in radians
        x = bond_length * np.sin(angle / 2)
        y = bond_length * np.cos(angle / 2)
        
        return [
            ("O", (0.0, 0.0, 0.0)),
            ("H", (x, y, 0.0)),
            ("H", (-x, y, 0.0))
        ]
        
    def _format_geometry(self, geometry):
        """Format geometry into string specification."""
        atom_spec = ""
        for atom in geometry:
            atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_spec.strip("; ")
        
    def _get_pyscf_mol(self, geometry):
        """Create a PySCF molecule object."""
        atom_spec = self._format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_spec
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        return mol
        
    def _get_qiskit_problem(self, geometry):
        """Create a Qiskit problem object."""
        atom_spec = self._format_geometry(geometry)
        driver = PySCFDriver(
            atom=atom_spec,
            unit=DistanceUnit.ANGSTROM,
            charge=self.charge,
            spin=self.spin,
            basis=self.basis
        )
        problem = driver.run()
        
        # Apply active space if defined
        if self.active_space:
            transformer = ActiveSpaceTransformer(
                num_electrons=self.active_space["num_electrons"],
                num_spatial_orbitals=self.active_space["num_spatial_orbitals"],
                active_orbitals=self.active_space["active_orbitals"]
            )
            problem = transformer.transform(problem)
            if self.verbose:
                print(f"Applied active space: {sum(problem.num_particles)} electrons in {problem.num_spatial_orbitals} orbitals")
                
        return problem
        
    def _get_mapper(self, problem):
        """Get appropriate mapper for the problem."""
        if self.mapper_type == "parity":
            return ParityMapper(num_particles=problem.num_particles)
        elif self.mapper_type == "bravyi-kitaev":
            return BravyiKitaevMapper()
        elif self.mapper_type == "jordan-wigner":
            return JordanWignerMapper()
        else:
            raise ValueError(f"Unsupported mapper type: {self.mapper_type}")
            
    def _get_optimizer(self):
        """Get appropriate optimizer."""
        if self.optimizer_name == "COBYLA":
            return COBYLA(**self.optimizer_kwargs)
        elif self.optimizer_name == "L_BFGS_B":
            return L_BFGS_B(**self.optimizer_kwargs)
        elif self.optimizer_name == "SLSQP":
            return SLSQP(**self.optimizer_kwargs)
        else:
            raise ValueError(f"Unsupported optimizer: {self.optimizer_name}")
            
    def run_vqe(self, problem, mapper):
        """Run VQE on the given problem."""
        # Create initial state and ansatz
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        # Use random initial parameters for better convergence
        np.random.seed(42)  # For reproducibility
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Create optimizer and estimator
        optimizer = self._get_optimizer()
        estimator = Estimator()  # CPU-based estimator
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Ansatz parameters: {ansatz.num_parameters}")
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
            
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        if self.verbose:
            print(f"VQE runtime: {runtime:.2f} seconds")
            
        # Interpret and return energy
        return problem.interpret(result).total_energies[0].real
        
    def run_reference_calcs(self, mol):
        """Run reference classical calculations."""
        results = {}
        
        # Run HF
        try:
            mf = scf.RHF(mol)
            results["hf"] = mf.kernel()
        except Exception as e:
            print(f"HF calculation failed: {e}")
            results["hf"] = None
            
        # Run FCI if molecule is small enough
        try:
            if self.molecule_type == "h2" or self.molecule_type == "h2o":
                mf_fci = scf.RHF(mol).run()
                cisolver = fci.FCI(mol, mf_fci.mo_coeff)
                results["fci"], _ = cisolver.kernel()
        except Exception as e:
            print(f"FCI calculation failed: {e}")
            results["fci"] = None
            
        # Run CASSCF if active space is defined
        try:
            if self.active_space:
                mf_cas = scf.RHF(mol).run()
                mycas = mcscf.CASSCF(
                    mf_cas,
                    self.active_space["num_spatial_orbitals"],
                    sum(self.active_space["num_electrons"])
                )
                results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF calculation failed: {e}")
            results["casscf"] = None
            
        return results
        
    def run_energy_scan(self):
        """Run energy scan over geometric parameters."""
        start, end, points = self.scan_range
        if points == 1:  # Single point calculation
            distances = [start]
        else:
            distances = np.linspace(start, end, points)
            
        vqe_energies, hf_energies, fci_energies, casscf_energies = [], [], [], []
        
        for i, dist in enumerate(distances):
            if self.verbose:
                print(f"\nPoint {i+1}/{len(distances)} at distance {dist:.2f} Å")
                
            # Get geometry and create molecule
            geometry = self.geometry_func(dist)
            mol = self._get_pyscf_mol(geometry)
            
            # Run reference calculations
            ref_results = self.run_reference_calcs(mol)
            hf_energies.append(ref_results.get("hf"))
            fci_energies.append(ref_results.get("fci"))
            casscf_energies.append(ref_results.get("casscf"))
            
            # Run VQE calculation
            try:
                problem = self._get_qiskit_problem(geometry)
                mapper = self._get_mapper(problem)
                vqe_energy = self.run_vqe(problem, mapper)
                vqe_energies.append(vqe_energy)
                
                # Print energies
                print(f"  Distance: {dist:.2f} Å | VQE: {vqe_energy:.6f} | HF: {ref_results.get('hf', 'N/A'):.6f}")
                if ref_results.get("fci") is not None:
                    print(f"  FCI: {ref_results.get('fci'):.6f}")
                if ref_results.get("casscf") is not None:
                    print(f"  CASSCF: {ref_results.get('casscf'):.6f}")
                    
            except Exception as e:
                print(f"VQE calculation failed at distance {dist:.2f} Å: {e}")
                vqe_energies.append(np.nan)
                
        # Store results
        self.results["distances"] = distances
        self.results["vqe"] = vqe_energies
        self.results["hf"] = hf_energies
        self.results["fci"] = fci_energies
        self.results["casscf"] = casscf_energies
        
        return self.results
        
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot. Run energy_scan first.")
            return
            
        plt.figure(figsize=(8, 6))
        
        # Plot VQE energies
        plt.plot(
            self.results["distances"],
            self.results["vqe"],
            'o-',
            label='VQE',
            color='limegreen',
            linewidth=2
        )
        
        # Plot HF energies
        if any(e is not None for e in self.results["hf"]):
            plt.plot(
                self.results["distances"],
                self.results["hf"],
                's-',
                label='HF',
                color='red',
                linewidth=2
            )
            
        # Plot FCI energies
        if any(e is not None for e in self.results["fci"]):
            plt.plot(
                self.results["distances"],
                self.results["fci"],
                'x--',
                label='FCI',
                color='black',
                linewidth=2
            )
            
        # Plot CASSCF energies
        if any(e is not None for e in self.results["casscf"]):
            plt.plot(
                self.results["distances"],
                self.results["casscf"],
                '*-',
                label='CASSCF',
                color='blue',
                linewidth=2
            )
            
        # Add plot details
        plt.xlabel(self.scan_label, fontsize=12)
        plt.ylabel("Energy (Hartree)", fontsize=12)
        plt.title(f"{self.title} ({self.basis} basis)", fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend(fontsize=10)
        plt.tight_layout()
        
        # Save or show
        if save_path:
            plt.savefig(save_path, dpi=300)
            print(f"Plot saved to {save_path}")
        else:
            plt.show()
            
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize."
            
        # Find minimum energies
        min_vqe_idx = np.nanargmin(self.results["vqe"])
        min_dist = self.results["distances"][min_vqe_idx]
        min_vqe = self.results["vqe"][min_vqe_idx]
        
        summary = f"\n{'=' * 30} RESULTS SUMMARY {'=' * 30}\n"
        summary += f"Molecule: {self.molecule_type.upper()}\n"
        summary += f"Basis set: {self.basis}\n"
        summary += f"Minimum VQE energy: {min_vqe:.8f} Hartree at {min_dist:.3f} Å\n"
        
        # Compare with reference values if available
        if self.results["hf"][min_vqe_idx] is not None:
            hf_energy = self.results["hf"][min_vqe_idx]
            summary += f"HF energy: {hf_energy:.8f} Hartree\n"
            summary += f"Correlation energy: {min_vqe - hf_energy:.8f} Hartree\n"
            
        if self.results["fci"] and self.results["fci"][min_vqe_idx] is not None:
            fci_energy = self.results["fci"][min_vqe_idx]
            summary += f"FCI energy: {fci_energy:.8f} Hartree\n"
            summary += f"VQE error vs FCI: {(min_vqe - fci_energy) * 1000:.3f} mHartree\n"
            
        if self.results["casscf"] and self.results["casscf"][min_vqe_idx] is not None:
            casscf_energy = self.results["casscf"][min_vqe_idx]
            summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
            summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
            
        summary += f"{'=' * 80}\n"
        return summary


# Example usage
if __name__ == "__main__":
    # Print timestamp
    print(f"Run started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Choose molecule type: "h2", "neon", or "h2o"
    molecule_type = "h2"  # Change this to simulate different molecules
    
    # Create simulator and run
    sim = MoleculeVQE(molecule_type=molecule_type, verbose=True)
    results = sim.run_energy_scan()
    
    # Plot results
    sim.plot_results()
    
    # Print summary
    print(sim.get_summary())

In [ ]:
"CH4"

In [7]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP

from pyscf import gto, scf, fci, mcscf

In [ ]:



class MethaneSim:
    """Specialized class for methane (CH₄) quantum simulation."""
    
    def __init__(self, verbose=True):
        """Initialize methane simulation with optimal settings."""
        self.verbose = verbose
        self.results = {}
        
        # Best configuration for methane
        self.basis = "6-31G"  # Good balance for CH₄
        self.charge = 0
        self.spin = 0
        self.active_space = {
            "num_electrons": (4, 4),  # 8 valence electrons
            "num_spatial_orbitals": 8,  # 8 spatial orbitals
            "active_orbitals": [1, 2, 3, 4, 5, 6, 7, 8]  # Skip C 1s core orbital
        }
        self.mapper_type = "parity"
        self.optimizer_name = "SLSQP"
        self.optimizer_kwargs = {"maxiter": 500}
        
        # Print configuration
        if self.verbose:
            self._print_config()
    
    def _print_config(self):
        """Print configuration details."""
        print(f"\n{'=' * 50}")
        print(f"Configuration for CH₄ (Methane)")
        print(f"{'=' * 50}")
        print(f"Basis set: {self.basis}")
        print(f"Charge: {self.charge}, Spin: {self.spin}")
        print(f"Active space: {self.active_space}")
        print(f"Mapper: {self.mapper_type}")
        print(f"Optimizer: {self.optimizer_name} with {self.optimizer_kwargs}")
        print(f"{'=' * 50}\n")
    
    def get_methane_geometry(self, scale_factor=1.0):
        """
        Generate tetrahedral methane geometry.
        
        Args:
            scale_factor: Scale the C-H bond length (1.0 = equilibrium ~1.09 Å)
        
        Returns:
            List of tuples with atom type and coordinates
        """
        # Equilibrium C-H bond length in methane is ~1.09 Å
        bond_length = 1.09 * scale_factor
        
        # Tetrahedral geometry
        a = bond_length / np.sqrt(3)
        
        return [
            ("C", (0.0, 0.0, 0.0)),
            ("H", (a, a, a)),
            ("H", (-a, -a, a)),
            ("H", (-a, a, -a)),
            ("H", (a, -a, -a))
        ]
    
    def _format_geometry(self, geometry):
        """Format geometry into string specification."""
        atom_spec = ""
        for atom in geometry:
            atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_spec.strip("; ")
    
    def _get_pyscf_mol(self, geometry):
        """Create a PySCF molecule object."""
        atom_spec = self._format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_spec
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        return mol
    
    def _get_qiskit_problem(self, geometry):
        """Create a Qiskit problem object."""
        atom_spec = self._format_geometry(geometry)
        driver = PySCFDriver(
            atom=atom_spec,
            unit=DistanceUnit.ANGSTROM,
            charge=self.charge,
            spin=self.spin,
            basis=self.basis
        )
        problem = driver.run()
        
        # Apply active space transformation
        transformer = ActiveSpaceTransformer(
            num_electrons=self.active_space["num_electrons"],
            num_spatial_orbitals=self.active_space["num_spatial_orbitals"],
            active_orbitals=self.active_space["active_orbitals"]
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Applied active space: {sum(problem.num_particles)} electrons in {problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe(self, problem):
        """Run VQE on the given problem."""
        # Create mapper (Parity with qubit reduction)
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create initial state and ansatz
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        # Use random initial parameters for better convergence
        np.random.seed(42)  # For reproducibility
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Create optimizer and estimator
        optimizer = SLSQP(**self.optimizer_kwargs)
        estimator = Estimator()  # CPU-based estimator
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Ansatz parameters: {ansatz.num_parameters}")
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        if self.verbose:
            print(f"VQE runtime: {runtime:.2f} seconds")
        
        # Interpret and return energy
        return problem.interpret(result).total_energies[0].real
    
    def run_reference_calcs(self, mol):
        """Run reference classical calculations."""
        results = {}
        
        # Run HF
        try:
            mf = scf.RHF(mol)
            results["hf"] = mf.kernel()
        except Exception as e:
            print(f"HF calculation failed: {e}")
            results["hf"] = None
        
        # Run CASSCF
        try:
            mf_cas = scf.RHF(mol).run()
            mycas = mcscf.CASSCF(
                mf_cas,
                self.active_space["num_spatial_orbitals"],
                sum(self.active_space["num_electrons"])
            )
            results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF calculation failed: {e}")
            results["casscf"] = None
        
        # FCI on methane is likely too expensive, but we could try MP2
        try:
            mf_mp2 = scf.RHF(mol).run()
            mp2_energy = mol.MP2().kernel()[0]
            results["mp2"] = mp2_energy + mf_mp2.e_tot
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = None
        
        return results
    
    def run_bond_scan(self, start=0.8, end=1.2, points=9):
        """
        Run energy scan over C-H bond length scaling.
        
        Args:
            start: Starting scale factor (0.8 = 80% of equilibrium)
            end: Ending scale factor (1.2 = 120% of equilibrium) 
            points: Number of points in the scan
        """
        scale_factors = np.linspace(start, end, points)
        distances = scale_factors * 1.09  # Convert scale factors to actual C-H distances
        
        vqe_energies, hf_energies, casscf_energies, mp2_energies = [], [], [], []
        
        for i, scale in enumerate(scale_factors):
            if self.verbose:
                print(f"\nPoint {i+1}/{len(scale_factors)} at C-H bond scaling {scale:.2f} (bond length: {distances[i]:.3f} Å)")
            
            # Get geometry and create molecule
            geometry = self.get_methane_geometry(scale)
            mol = self._get_pyscf_mol(geometry)
            
            # Run reference calculations
            ref_results = self.run_reference_calcs(mol)
            hf_energies.append(ref_results.get("hf"))
            casscf_energies.append(ref_results.get("casscf"))
            mp2_energies.append(ref_results.get("mp2"))
            
            # Run VQE calculation
            try:
                problem = self._get_qiskit_problem(geometry)
                vqe_energy = self.run_vqe(problem)
                vqe_energies.append(vqe_energy)
                
                # Print energies
                print(f"  Scale: {scale:.2f} | C-H dist: {distances[i]:.3f} Å | VQE: {vqe_energy:.6f} | HF: {ref_results.get('hf', 'N/A'):.6f}")
                if ref_results.get("casscf") is not None:
                    print(f"  CASSCF: {ref_results.get('casscf'):.6f}")
                if ref_results.get("mp2") is not None:
                    print(f"  MP2: {ref_results.get('mp2'):.6f}")
            
            except Exception as e:
                print(f"VQE calculation failed at scale {scale:.2f}: {e}")
                vqe_energies.append(np.nan)
        
        # Store results
        self.results["distances"] = distances
        self.results["vqe"] = vqe_energies
        self.results["hf"] = hf_energies
        self.results["casscf"] = casscf_energies
        self.results["mp2"] = mp2_energies
        
        return self.results
    
    def run_equilibrium(self):
        """Run single point calculation at equilibrium geometry."""
        print("\nRunning single point calculation at equilibrium geometry...")
        
        # Get geometry and create molecule
        geometry = self.get_methane_geometry(1.0)
        mol = self._get_pyscf_mol(geometry)
        
        # Run reference calculations
        ref_results = self.run_reference_calcs(mol)
        
        # Run VQE calculation
        try:
            problem = self._get_qiskit_problem(geometry)
            vqe_energy = self.run_vqe(problem)
            
            # Store results
            self.results["equilibrium"] = {
                "vqe": vqe_energy,
                "hf": ref_results.get("hf"),
                "casscf": ref_results.get("casscf"),
                "mp2": ref_results.get("mp2")
            }
            
            print(f"\nEquilibrium CH₄ energies:")
            print(f"  VQE: {vqe_energy:.6f} Hartree")
            print(f"  HF: {ref_results.get('hf', 'N/A'):.6f} Hartree")
            if ref_results.get("casscf") is not None:
                print(f"  CASSCF: {ref_results.get('casscf'):.6f} Hartree")
            if ref_results.get("mp2") is not None:
                print(f"  MP2: {ref_results.get('mp2'):.6f} Hartree")
            
            return self.results["equilibrium"]
        
        except Exception as e:
            print(f"VQE calculation failed: {e}")
            return None
    
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if "distances" not in self.results:
            print("No bond scan results to plot. Run bond_scan first.")
            return
        
        plt.figure(figsize=(8, 6))
        
        # Plot VQE energies
        plt.plot(
            self.results["distances"],
            self.results["vqe"],
            'o-',
            label='VQE',
            color='limegreen',
            linewidth=2
        )
        
        # Plot HF energies
        if any(e is not None for e in self.results["hf"]):
            plt.plot(
                self.results["distances"],
                self.results["hf"],
                's-',
                label='HF',
                color='red',
                linewidth=2
            )
        
        # Plot CASSCF energies
        if any(e is not None for e in self.results["casscf"]):
            plt.plot(
                self.results["distances"],
                self.results["casscf"],
                '*-',
                label='CASSCF',
                color='blue',
                linewidth=2
            )
        
        # Plot MP2 energies
        if any(e is not None for e in self.results["mp2"]):
            plt.plot(
                self.results["distances"],
                self.results["mp2"],
                'x-',
                label='MP2',
                color='purple',
                linewidth=2
            )
        
        # Add plot details
        plt.xlabel("C-H Bond Length (Å)", fontsize=12)
        plt.ylabel("Energy (Hartree)", fontsize=12)
        plt.title(f"CH₄ Potential Energy Curve ({self.basis} basis)", fontsize=14)
        plt.grid(True, alpha=0.3)
        plt.legend(fontsize=10)
        plt.tight_layout()
        
        # Save or show
        if save_path:
            plt.savefig(save_path, dpi=300)
            print(f"Plot saved to {save_path}")
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if "equilibrium" in self.results:
            # Single point summary
            eq = self.results["equilibrium"]
            
            summary = f"\n{'=' * 30} CH₄ RESULTS SUMMARY {'=' * 30}\n"
            summary += f"Basis set: {self.basis}\n"
            summary += f"VQE energy: {eq['vqe']:.8f} Hartree\n"
            
            if eq["hf"] is not None:
                summary += f"HF energy: {eq['hf']:.8f} Hartree\n"
                summary += f"Correlation energy: {eq['vqe'] - eq['hf']:.8f} Hartree\n"
            
            if eq["casscf"] is not None:
                summary += f"CASSCF energy: {eq['casscf']:.8f} Hartree\n"
                summary += f"VQE error vs CASSCF: {(eq['vqe'] - eq['casscf']) * 1000:.3f} mHartree\n"
            
            if eq["mp2"] is not None:
                summary += f"MP2 energy: {eq['mp2']:.8f} Hartree\n"
                summary += f"VQE error vs MP2: {(eq['vqe'] - eq['mp2']) * 1000:.3f} mHartree\n"
            
        elif "distances" in self.results:
            # Bond scan summary
            min_vqe_idx = np.nanargmin(self.results["vqe"])
            min_dist = self.results["distances"][min_vqe_idx]
            min_vqe = self.results["vqe"][min_vqe_idx]
            
            summary = f"\n{'=' * 30} CH₄ RESULTS SUMMARY {'=' * 30}\n"
            summary += f"Basis set: {self.basis}\n"
            summary += f"Minimum VQE energy: {min_vqe:.8f} Hartree at C-H bond length {min_dist:.3f} Å\n"
            
            if self.results["hf"][min_vqe_idx] is not None:
                hf_energy = self.results["hf"][min_vqe_idx]
                summary += f"HF energy: {hf_energy:.8f} Hartree\n"
                summary += f"Correlation energy: {min_vqe - hf_energy:.8f} Hartree\n"
            
            if self.results["casscf"] and self.results["casscf"][min_vqe_idx] is not None:
                casscf_energy = self.results["casscf"][min_vqe_idx]
                summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
                summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
            
            if self.results["mp2"] and self.results["mp2"][min_vqe_idx] is not None:
                mp2_energy = self.results["mp2"][min_vqe_idx]
                summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
                summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
        else:
            return "No results to summarize."
        
        summary += f"{'=' * 80}\n"
        return summary


# Example usage
if __name__ == "__main__":
    # Print timestamp
    print(f"Run started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create methane simulator
    sim = MethaneSim(verbose=True)
    
    # Option 1: Run single point at equilibrium geometry
    sim.run_equilibrium()
    
    # Option 2: Run bond scan
    # results = sim.run_bond_scan(start=0.8, end=1.2, points=5)
    # sim.plot_results()
    
    # Print summary
    print(sim.get_summary())

In [ ]:
"N2"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B, SLSQP

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

# PySCF for classical references
from pyscf import gto, scf, fci, mcscf

# --- Best settings for N₂ ---
BASIS = "6-31g"  # Better basis for N₂ (compromise between accuracy and cost)
# Active space: focus on 2p orbitals involved in triple bond + lone pairs
ACTIVE_ELECTRONS = (5, 5)  # 10 valence electrons (5 alpha, 5 beta)
ACTIVE_SPATIAL_ORBITALS = 8  # Including 2s and 2p orbitals
ACTIVE_ORBITALS = [2, 3, 4, 5, 6, 7, 8, 9]  # Skip 1s core orbitals (indices 0,1)
MAX_ITER = 500  # More iterations for N₂'s complex landscape

# Bond distances to scan (N-N bond in Angstrom)
# Include more points around equilibrium (1.098 Å)
DISTS = np.concatenate([
    np.linspace(0.9, 1.3, 9),   # More points around equilibrium
    np.linspace(1.4, 2.6, 7)    # Fewer points in dissociation region
])

# --- Helper functions ---
def build_pyscf_mol(distance, basis=BASIS):
    # Linear molecule along z-axis: N at -d/2 and +d/2
    atom = f"N 0 0 {-distance/2}; N 0 0 {distance/2}"
    mol = gto.Mole()
    mol.atom = atom
    mol.basis = basis
    mol.charge = 0
    mol.spin = 0
    mol.build()
    return mol

def run_references(mol):
    """Run multiple reference calculations."""
    results = {}
    
    # RHF
    try:
        mf = scf.RHF(mol)
        results["rhf"] = mf.kernel()
        mf_obj = mf
    except Exception as e:
        print(f"RHF failed: {e}")
        results["rhf"] = np.nan
        mf_obj = None
    
    # CASSCF with same active space as VQE
    try:
        mf_cas = scf.RHF(mol).run()
        mycas = mcscf.CASSCF(mf_cas, 
                            nelecas=sum(ACTIVE_ELECTRONS), 
                            ncas=ACTIVE_SPATIAL_ORBITALS)
        results["casscf"] = mycas.kernel()[0]
    except Exception as e:
        print(f"CASSCF failed: {e}")
        results["casscf"] = np.nan
    
    # Try FCI in active space if feasible
    if mf_obj is not None:
        try:
            # Create a smaller molecule with just the active space
            cisolver = fci.FCI(mol, mf_obj.mo_coeff)
            e, _ = cisolver.kernel()
            results["fci"] = e
        except Exception as e:
            print(f"FCI failed (likely too large): {e}")
            results["fci"] = np.nan
    
    return results

def build_qiskit_problem(distance, basis=BASIS):
    """Build Qiskit Nature problem with proper active space."""
    atom = f"N 0 0 {-distance/2}; N 0 0 {distance/2}"
    driver = PySCFDriver(atom=atom, basis=basis, unit=DistanceUnit.ANGSTROM,
                         charge=0, spin=0)
    problem = driver.run()
    
    # Print full system information
    print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
          f"{problem.num_particles} electrons")
    
    # Apply active space transformer with explicit orbital selection
    transformer = ActiveSpaceTransformer(
        num_electrons=ACTIVE_ELECTRONS,
        num_spatial_orbitals=ACTIVE_SPATIAL_ORBITALS,
        active_orbitals=ACTIVE_ORBITALS  # Skip core 1s orbitals
    )
    problem_reduced = transformer.transform(problem)
    
    print(f"Active space: {sum(problem_reduced.num_particles)} electrons in "
          f"{problem_reduced.num_spatial_orbitals} orbitals")
    
    return problem_reduced

def run_vqe(problem):
    """Run VQE with improved settings."""
    # Use parity mapper with symmetry preservation
    mapper = ParityMapper(num_particles=problem.num_particles)
    
    # Create initial state and ansatz
    init_state = HartreeFock(problem.num_spatial_orbitals, 
                             problem.num_particles, 
                             mapper)
    ansatz = UCCSD(problem.num_spatial_orbitals, 
                  problem.num_particles, 
                  mapper,
                  initial_state=init_state)
    
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # Better initialization - small random values
    np.random.seed(42)  # For reproducibility
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # Use L-BFGS-B optimizer - better for N₂'s landscape
    optimizer = L_BFGS_B(maxiter=MAX_ITER)
    
    # Create CPU estimator
    estimator = Estimator()
    
    # Create and run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    
    # Run VQE and time it
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    runtime = time.time() - start_time
    
    # Get interpreted energy
    energy = problem.interpret(result).total_energies[0].real
    
    print(f"VQE completed in {runtime:.1f} seconds")
    print(f"VQE energy: {energy:.8f} Hartree")
    print(f"Optimizer converged: {result.optimizer_evals} iterations")
    
    return energy, result, qubit_op

# --- Main sweep ---
print(f"\n{'='*50}")
print(f"N₂ Potential Energy Surface with VQE")
print(f"Basis: {BASIS}")
print(f"Active space: {sum(ACTIVE_ELECTRONS)}e, {ACTIVE_SPATIAL_ORBITALS}o")
print(f"{'='*50}")

vqe_energies = []
rhf_energies = []
casscf_energies = []
fci_energies = []

for d in DISTS:
    print(f"\n{'-'*30}")
    print(f"Distance: {d:.3f} Å")
    print(f"{'-'*30}")
    
    # Run classical references
    try:
        mol = build_pyscf_mol(d)
        ref_results = run_references(mol)
        rhf_energies.append(ref_results.get("rhf", np.nan))
        casscf_energies.append(ref_results.get("casscf", np.nan))
        fci_energies.append(ref_results.get("fci", np.nan))
    except Exception as e:
        print(f"Reference calculations failed: {e}")
        rhf_energies.append(np.nan)
        casscf_energies.append(np.nan)
        fci_energies.append(np.nan)
    
    # Run VQE
    try:
        problem = build_qiskit_problem(d)
        vqe_e, _, _ = run_vqe(problem)
        vqe_energies.append(vqe_e)
    except Exception as e:
        print(f"VQE calculation failed: {e}")
        vqe_energies.append(np.nan)

# --- Calculate N₂ dissociation energy ---
try:
    # Find minimum energy
    min_idx = np.nanargmin(vqe_energies)
    min_energy = vqe_energies[min_idx]
    eq_distance = DISTS[min_idx]
    
    # Energy at large separation (approximates 2 x N atom)
    dissoc_idx = -1  # Last point (largest separation)
    dissoc_energy = vqe_energies[dissoc_idx]
    
    # Dissociation energy
    de = min_energy - dissoc_energy
    print(f"\nDissociation energy: {-de:.6f} Hartree ({-de*627.5:.2f} kcal/mol)")
except Exception as e:
    print(f"Could not calculate dissociation energy: {e}")

# --- Plotting ---
plt.figure(figsize=(10, 6))

# Plot VQE energies
plt.plot(DISTS, vqe_energies, 'o-', label='VQE (active space)', 
         color='limegreen', linewidth=2, markersize=8)

# Plot RHF energies
if any(~np.isnan(rhf_energies)):
    plt.plot(DISTS, rhf_energies, 's--', label='RHF', 
             color='red', linewidth=2, markersize=6)

# Plot CASSCF energies
if any(~np.isnan(casscf_energies)):
    plt.plot(DISTS, casscf_energies, 'd-', label='CASSCF', 
             color='blue', linewidth=2, markersize=6)

# Plot FCI energies if available
if any(~np.isnan(fci_energies)):
    plt.plot(DISTS, fci_energies, 'x--', label='FCI', 
             color='black', linewidth=2, markersize=6)

# Plot styling
plt.xlabel("N–N Distance (Å)", fontsize=14)
plt.ylabel("Energy (Hartree)", fontsize=14)
plt.title(f"N$_2$ Potential Energy Curve ({BASIS} basis)", fontsize=16)
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()

# Add text box with summary
try:
    textstr = f"Equilibrium distance: {eq_distance:.3f} Å\n"
    textstr += f"VQE minimum energy: {min_energy:.6f} Ha\n"
    textstr += f"Dissociation energy: {-de*627.5:.1f} kcal/mol"
    
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
    plt.text(0.05, 0.05, textstr, transform=plt.gca().transAxes, 
             fontsize=12, verticalalignment='bottom', bbox=props)
except:
    pass

plt.savefig('n2_potential_curve.png', dpi=300)
plt.show()

# --- Report minimum from VQE sweep ---
if np.all(np.isnan(vqe_energies)):
    print("\nNo successful VQE energies calculated.")
else:
    idx = int(np.nanargmin(vqe_energies))
    print("\n=== Minimum VQE result (from sweep) ===")
    print(f"Equilibrium distance: {DISTS[idx]:.3f} Å")
    print(f"VQE energy: {vqe_energies[idx]:.8f} Hartree")
    
    # Compare with references
    if not np.isnan(rhf_energies[idx]):
        print(f"RHF energy: {rhf_energies[idx]:.8f} Hartree")
        print(f"Correlation energy: {vqe_energies[idx] - rhf_energies[idx]:.8f} Hartree")
    
    if not np.isnan(casscf_energies[idx]):
        print(f"CASSCF energy: {casscf_energies[idx]:.8f} Hartree")
        print(f"VQE vs CASSCF: {(vqe_energies[idx] - casscf_energies[idx])*1000:.3f} mHartree")

In [ ]:
'O2'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper, BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, L_BFGS_B

from pyscf import gto, scf, mcscf

# --- Best settings for O₂ ---
BASIS = "6-31g"  # Better basis for O₂
# Active space: 12e in 10o (2s and 2p orbitals from each O)
ACTIVE_ELECTRONS = (7, 5)  # 12 electrons with 2 unpaired (7 alpha, 5 beta)
ACTIVE_SPATIAL_ORBITALS = 10  # 10 spatial orbitals (2s, 2p from each O)
# Skip 1s core orbitals (indices 0,1)
ACTIVE_ORBITALS = list(range(2, 12))
MAX_ITER = 500  # More iterations for O₂'s complex landscape

class O2Simulator:
    """Class for simulating O₂ using VQE with best practices."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("Oxygen (O₂) VQE Simulation with Best Practices")
            print(f"{'='*50}")
            print(f"Basis: {BASIS}")
            print(f"Active Space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_SPATIAL_ORBITALS}o")
            print(f"Spin: Triplet (2 unpaired electrons)")
            print(f"{'='*50}\n")
    
    def get_o2_geometry(self, distance):
        """Generate O₂ coordinates with given bond distance."""
        return [
            ("O", (0.0, 0.0, -distance/2)),
            ("O", (0.0, 0.0, distance/2))
        ]
    
    def format_geometry(self, geometry):
        """Convert geometry to string format."""
        atom_str = ""
        for atom in geometry:
            atom_str += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_str.strip("; ")
    
    def build_pyscf_mol(self, geometry):
        """Create PySCF molecule object."""
        atom_str = self.format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_str
        mol.basis = BASIS
        mol.charge = 0
        mol.spin = 2  # Triplet state (2 unpaired electrons)
        mol.build()
        return mol
    
    def run_uhf(self, mol):
        """Run UHF calculation."""
        mf = scf.UHF(mol)
        return mf.kernel()
    
    def run_casscf(self, mol):
        """Run CASSCF with matching active space."""
        mf = scf.UHF(mol).run()
        mc = mcscf.CASSCF(mf, 
                          ncas=ACTIVE_SPATIAL_ORBITALS, 
                          nelecas=sum(ACTIVE_ELECTRONS))
        return mc.kernel()[0]
    
    def build_qiskit_problem(self, geometry):
        """Build Qiskit Nature problem with proper active space."""
        atom_str = self.format_geometry(geometry)
        
        # Create driver and run
        driver = PySCFDriver(
            atom=atom_str, 
            basis=BASIS, 
            unit=DistanceUnit.ANGSTROM, 
            charge=0, 
            spin=2  # Triplet state
        )
        problem = driver.run()
        
        # Print full system information
        if self.verbose:
            print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
                  f"{problem.num_particles} electrons")
        
        # Apply active space transformation with explicit orbital selection
        transformer = ActiveSpaceTransformer(
            num_electrons=ACTIVE_ELECTRONS,
            num_spatial_orbitals=ACTIVE_SPATIAL_ORBITALS,
            active_orbitals=ACTIVE_ORBITALS
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Active space: {sum(problem.num_particles)} electrons in "
                  f"{problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe(self, problem):
        """Run VQE with optimal settings."""
        # Use parity mapper with symmetry reduction
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create initial state and ansatz
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        if self.verbose:
            print(f"UCCSD parameters: {ansatz.num_parameters}")
        
        # Better initialization - small random values with seed for reproducibility
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Use SLSQP optimizer - works well for O₂
        optimizer = SLSQP(maxiter=MAX_ITER)
        
        # Create CPU estimator
        estimator = Estimator()
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE and time it
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get interpreted energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.1f} seconds")
            print(f"VQE energy: {energy:.8f} Hartree")
            print(f"Optimizer evaluations: {result.optimizer_evals}")
        
        return energy, result
    
    def run_energy_scan(self, distances=None):
        """Run energy scan over O-O bond distances."""
        if distances is None:
            # Default distances: more points near equilibrium (~1.21 Å)
            distances = np.concatenate([
                np.linspace(1.0, 1.4, 9),  # Dense near equilibrium
                np.linspace(1.5, 2.6, 6)   # Sparser in dissociation region
            ])
        
        vqe_energies = []
        uhf_energies = []
        casscf_energies = []
        
        for i, d in enumerate(distances):
            print(f"\n{'-'*30}")
            print(f"Distance: {d:.3f} Å ({i+1}/{len(distances)})")
            print(f"{'-'*30}")
            
            # Generate geometry
            geometry = self.get_o2_geometry(d)
            
            # Run classical references
            try:
                mol = self.build_pyscf_mol(geometry)
                uhf_energy = self.run_uhf(mol)
                uhf_energies.append(uhf_energy)
                print(f"UHF energy: {uhf_energy:.8f} Hartree")
            except Exception as e:
                print(f"UHF calculation failed: {e}")
                uhf_energies.append(np.nan)
            
            # Run CASSCF reference
            try:
                casscf_energy = self.run_casscf(mol)
                casscf_energies.append(casscf_energy)
                print(f"CASSCF energy: {casscf_energy:.8f} Hartree")
            except Exception as e:
                print(f"CASSCF calculation failed: {e}")
                casscf_energies.append(np.nan)
            
            # Run VQE
            try:
                problem = self.build_qiskit_problem(geometry)
                vqe_energy, _ = self.run_vqe(problem)
                vqe_energies.append(vqe_energy)
                print(f"VQE energy: {vqe_energy:.8f} Hartree")
            except Exception as e:
                print(f"VQE calculation failed: {e}")
                vqe_energies.append(np.nan)
        
        # Store results
        self.results = {
            'distances': distances,
            'vqe': vqe_energies,
            'uhf': uhf_energies,
            'casscf': casscf_energies
        }
        
        return self.results
    
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot. Run energy_scan first.")
            return
        
        plt.figure(figsize=(10, 6))
        
        # Plot VQE energies
        plt.plot(
            self.results['distances'],
            self.results['vqe'],
            'o-',
            label='VQE (active space)',
            color='limegreen',
            linewidth=2,
            markersize=8
        )
        
        # Plot UHF energies
        if any(~np.isnan(self.results['uhf'])):
            plt.plot(
                self.results['distances'],
                self.results['uhf'],
                's--',
                label='UHF',
                color='red',
                linewidth=2,
                markersize=6
            )
        
        # Plot CASSCF energies
        if any(~np.isnan(self.results['casscf'])):
            plt.plot(
                self.results['distances'],
                self.results['casscf'],
                'd-',
                label='CASSCF',
                color='blue',
                linewidth=2,
                markersize=6
            )
        
        # Plot styling
        plt.xlabel("O–O Distance (Å)", fontsize=14)
        plt.ylabel("Energy (Hartree)", fontsize=14)
        plt.title(f"O$_2$ Potential Energy Curve ({BASIS} basis)", fontsize=16)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Add text box with equilibrium information
        try:
            vqe_min_idx = np.nanargmin(self.results['vqe'])
            eq_distance = self.results['distances'][vqe_min_idx]
            min_energy = self.results['vqe'][vqe_min_idx]
            
            # Calculate dissociation energy (approximation)
            dissoc_idx = -1  # Last point (largest separation)
            dissoc_energy = self.results['vqe'][dissoc_idx]
            de = min_energy - dissoc_energy
            
            textstr = f"Equilibrium distance: {eq_distance:.3f} Å\n"
            textstr += f"VQE minimum energy: {min_energy:.6f} Ha\n"
            textstr += f"Dissociation energy: {-de*627.5:.1f} kcal/mol"
            
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
            plt.text(0.05, 0.05, textstr, transform=plt.gca().transAxes, 
                     fontsize=12, verticalalignment='bottom', bbox=props)
        except:
            pass
        
        if save_path:
            plt.savefig(save_path, dpi=300)
            print(f"Plot saved to {save_path}")
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize."
        
        try:
            # Find minimum energies
            vqe_min_idx = np.nanargmin(self.results['vqe'])
            eq_distance = self.results['distances'][vqe_min_idx]
            min_vqe = self.results['vqe'][vqe_min_idx]
            
            summary = f"\n{'=' * 30} O₂ RESULTS SUMMARY {'=' * 30}\n"
            summary += f"Basis set: {BASIS}\n"
            summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_SPATIAL_ORBITALS}o\n"
            summary += f"Equilibrium distance: {eq_distance:.3f} Å\n"
            summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
            
            # Compare with reference values
            if not np.isnan(self.results['uhf'][vqe_min_idx]):
                uhf_energy = self.results['uhf'][vqe_min_idx]
                summary += f"UHF energy: {uhf_energy:.8f} Hartree\n"
                summary += f"Correlation energy: {min_vqe - uhf_energy:.8f} Hartree\n"
            
            if not np.isnan(self.results['casscf'][vqe_min_idx]):
                casscf_energy = self.results['casscf'][vqe_min_idx]
                summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
                summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
            
            # Calculate dissociation energy
            try:
                dissoc_idx = -1  # Last point (largest separation)
                dissoc_energy = self.results['vqe'][dissoc_idx]
                de = min_vqe - dissoc_energy
                summary += f"Dissociation energy: {-de:.6f} Hartree ({-de*627.5:.2f} kcal/mol)\n"
            except:
                pass
            
            summary += f"{'=' * 80}\n"
            return summary
        except Exception as e:
            return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"O₂ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create simulator
    sim = O2Simulator(verbose=True)
    
    # Run energy scan
    results = sim.run_energy_scan()
    
    # Plot results
    sim.plot_results(save_path="o2_potential_curve.png")
    
    # Print summary
    print(sim.get_summary())

In [ ]:
"C2H2"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, L_BFGS_B

from pyscf import gto, scf, mcscf, mp

# --- Best settings for C₂H₂ ---
BASIS = "6-31g"  # Better basis for acetylene
# Active space: focus on triple bond and C-H bonds
ACTIVE_ELECTRONS = (5, 5)  # 10 valence electrons (5 alpha, 5 beta)
ACTIVE_SPATIAL_ORBITALS = 10  # 10 spatial orbitals for valence space
# Skip C 1s core orbitals (first 2 orbitals for each C)
ACTIVE_ORBITALS = list(range(4, 14))  # Skip 4 core orbitals
MAX_ITER = 400  # More iterations for C₂H₂'s complex landscape

class AcetyleneSimulator:
    """Class for simulating acetylene (C₂H₂) using VQE with best practices."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("Acetylene (C₂H₂) VQE Simulation with Best Practices")
            print(f"{'='*50}")
            print(f"Basis: {BASIS}")
            print(f"Active Space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_SPATIAL_ORBITALS}o")
            print(f"{'='*50}\n")
    
    def get_c2h2_geometry(self, cc_distance=1.2, ch_distance=1.06):
        """
        Generate linear acetylene geometry.
        
        Args:
            cc_distance: C≡C bond length in Å (default: 1.2)
            ch_distance: C-H bond length in Å (default: 1.06)
            
        Returns:
            List of tuples with atom type and coordinates
        """
        return [
            ("C", (0.0, 0.0, -cc_distance/2)),
            ("C", (0.0, 0.0, cc_distance/2)),
            ("H", (0.0, 0.0, -cc_distance/2 - ch_distance)),
            ("H", (0.0, 0.0, cc_distance/2 + ch_distance))
        ]
    
    def format_geometry(self, geometry):
        """Convert geometry to string format."""
        atom_str = ""
        for atom in geometry:
            atom_str += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_str.strip("; ")
    
    def build_pyscf_mol(self, geometry):
        """Create PySCF molecule object."""
        atom_str = self.format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_str
        mol.basis = BASIS
        mol.charge = 0
        mol.spin = 0
        mol.build()
        return mol
    
    def run_reference_calcs(self, mol):
        """Run reference classical calculations."""
        results = {}
        
        # Run RHF
        try:
            mf = scf.RHF(mol)
            results["rhf"] = mf.kernel()
            mf_obj = mf
        except Exception as e:
            print(f"RHF calculation failed: {e}")
            results["rhf"] = np.nan
            mf_obj = None
        
        # Run MP2 for dynamic correlation
        try:
            mp2 = mp.MP2(mf_obj)
            mp2.kernel()
            # MP2 correlation energy + RHF energy
            results["mp2"] = mp2.e_corr + results["rhf"]
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = np.nan
        
        # Run CASSCF with same active space as VQE
        try:
            mf_cas = scf.RHF(mol).run()
            mycas = mcscf.CASSCF(mf_cas, 
                                ncas=ACTIVE_SPATIAL_ORBITALS, 
                                nelecas=sum(ACTIVE_ELECTRONS))
            results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF calculation failed: {e}")
            results["casscf"] = np.nan
        
        return results
    
    def build_qiskit_problem(self, geometry):
        """Build Qiskit Nature problem with proper active space."""
        atom_str = self.format_geometry(geometry)
        
        # Create driver and run
        driver = PySCFDriver(
            atom=atom_str, 
            basis=BASIS, 
            unit=DistanceUnit.ANGSTROM, 
            charge=0, 
            spin=0
        )
        problem = driver.run()
        
        # Print full system information
        if self.verbose:
            print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
                  f"{problem.num_particles} electrons")
        
        # Apply active space transformation with explicit orbital selection
        transformer = ActiveSpaceTransformer(
            num_electrons=ACTIVE_ELECTRONS,
            num_spatial_orbitals=ACTIVE_SPATIAL_ORBITALS,
            active_orbitals=ACTIVE_ORBITALS  # Skip core orbitals
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Active space: {sum(problem.num_particles)} electrons in "
                  f"{problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe(self, problem):
        """Run VQE with optimal settings."""
        # Use parity mapper with symmetry reduction
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create initial state and ansatz
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        if self.verbose:
            print(f"UCCSD parameters: {ansatz.num_parameters}")
        
        # Better initialization - small random values with seed for reproducibility
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Use SLSQP optimizer - works well for C₂H₂
        optimizer = SLSQP(maxiter=MAX_ITER)
        
        # Create CPU estimator
        estimator = Estimator()
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE and time it
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get interpreted energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.1f} seconds")
            print(f"VQE energy: {energy:.8f} Hartree")
            print(f"Optimizer evaluations: {result.optimizer_evals}")
        
        return energy, result
    
    def run_cc_scan(self, cc_distances=None, ch_distance=1.06):
        """
        Run energy scan over C≡C bond distances.
        
        Args:
            cc_distances: List of C≡C distances to scan (default: around equilibrium)
            ch_distance: Fixed C-H bond length (default: 1.06 Å)
        """
        if cc_distances is None:
            # Default distances: focused around equilibrium (~1.2 Å)
            cc_distances = np.linspace(1.0, 1.6, 9)
        
        vqe_energies = []
        rhf_energies = []
        mp2_energies = []
        casscf_energies = []
        
        for i, cc_d in enumerate(cc_distances):
            print(f"\n{'-'*30}")
            print(f"C≡C Distance: {cc_d:.3f} Å ({i+1}/{len(cc_distances)})")
            print(f"{'-'*30}")
            
            # Generate geometry
            geometry = self.get_c2h2_geometry(cc_distance=cc_d, ch_distance=ch_distance)
            
            # Run classical references
            mol = self.build_pyscf_mol(geometry)
            ref_results = self.run_reference_calcs(mol)
            rhf_energies.append(ref_results.get("rhf"))
            mp2_energies.append(ref_results.get("mp2"))
            casscf_energies.append(ref_results.get("casscf"))
            
            # Run VQE
            try:
                problem = self.build_qiskit_problem(geometry)
                vqe_energy, _ = self.run_vqe(problem)
                vqe_energies.append(vqe_energy)
                
                # Print comparison
                print("\nEnergy comparison at CC distance = {:.3f} Å:".format(cc_d))
                print(f"  VQE:    {vqe_energy:.8f} Hartree")
                print(f"  RHF:    {ref_results.get('rhf', 'N/A')}")
                if ref_results.get("mp2") is not None:
                    print(f"  MP2:    {ref_results.get('mp2'):.8f} Hartree")
                if ref_results.get("casscf") is not None:
                    print(f"  CASSCF: {ref_results.get('casscf'):.8f} Hartree")
            
            except Exception as e:
                print(f"VQE calculation failed: {e}")
                vqe_energies.append(np.nan)
        
        # Store results
        self.results = {
            'cc_distances': cc_distances,
            'vqe': vqe_energies,
            'rhf': rhf_energies,
            'mp2': mp2_energies,
            'casscf': casscf_energies
        }
        
        return self.results
    
    def run_ch_scan(self, cc_distance=1.2, ch_distances=None):
        """
        Run energy scan over C-H bond distances with fixed C≡C distance.
        
        Args:
            cc_distance: Fixed C≡C bond length (default: 1.2 Å)
            ch_distances: List of C-H distances to scan
        """
        if ch_distances is None:
            # Default: scan around equilibrium C-H distance
            ch_distances = np.linspace(0.9, 1.3, 7)
        
        vqe_energies = []
        rhf_energies = []
        casscf_energies = []
        
        for i, ch_d in enumerate(ch_distances):
            print(f"\n{'-'*30}")
            print(f"C-H Distance: {ch_d:.3f} Å ({i+1}/{len(ch_distances)})")
            print(f"{'-'*30}")
            
            # Generate geometry
            geometry = self.get_c2h2_geometry(cc_distance=cc_distance, ch_distance=ch_d)
            
            # Run classical references and VQE
            # (Implementation similar to run_cc_scan)
            # ...
            
        # Store and return results
        # ...
    
    def plot_results(self, save_path=None, scan_type='cc'):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot. Run a scan first.")
            return
        
        plt.figure(figsize=(10, 6))
        
        if scan_type == 'cc':
            distances = self.results['cc_distances']
            x_label = "C≡C Distance (Å)"
            title = f"C₂H₂ C≡C Bond Scan ({BASIS} basis)"
        else:
            distances = self.results['ch_distances']
            x_label = "C-H Distance (Å)"
            title = f"C₂H₂ C-H Bond Scan ({BASIS} basis)"
        
        # Plot VQE energies
        plt.plot(
            distances,
            self.results['vqe'],
            'o-',
            label='VQE (active space)',
            color='limegreen',
            linewidth=2,
            markersize=8
        )
        
        # Plot RHF energies
        if any(~np.isnan(self.results['rhf'])):
            plt.plot(
                distances,
                self.results['rhf'],
                's--',
                label='RHF',
                color='red',
                linewidth=2,
                markersize=6
            )
        
        # Plot MP2 energies if available
        if 'mp2' in self.results and any(~np.isnan(self.results['mp2'])):
            plt.plot(
                distances,
                self.results['mp2'],
                'x-',
                label='MP2',
                color='purple',
                linewidth=2,
                markersize=6
            )
        
        # Plot CASSCF energies
        if any(~np.isnan(self.results['casscf'])):
            plt.plot(
                distances,
                self.results['casscf'],
                'd-',
                label='CASSCF',
                color='blue',
                linewidth=2,
                markersize=6
            )
        
        # Plot styling
        plt.xlabel(x_label, fontsize=14)
        plt.ylabel("Energy (Hartree)", fontsize=14)
        plt.title(title, fontsize=16)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Add text box with equilibrium information
        try:
            vqe_min_idx = np.nanargmin(self.results['vqe'])
            eq_distance = distances[vqe_min_idx]
            min_energy = self.results['vqe'][vqe_min_idx]
            
            textstr = f"Equilibrium distance: {eq_distance:.3f} Å\n"
            textstr += f"VQE minimum energy: {min_energy:.6f} Ha"
            
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
            plt.text(0.05, 0.05, textstr, transform=plt.gca().transAxes, 
                     fontsize=12, verticalalignment='bottom', bbox=props)
        except:
            pass
        
        if save_path:
            plt.savefig(save_path, dpi=300)
            print(f"Plot saved to {save_path}")
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize."
        
        try:
            # Find minimum energies
            vqe_min_idx = np.nanargmin(self.results['vqe'])
            
            if 'cc_distances' in self.results:
                distances = self.results['cc_distances']
                distance_type = "C≡C"
            else:
                distances = self.results['ch_distances']
                distance_type = "C-H"
            
            eq_distance = distances[vqe_min_idx]
            min_vqe = self.results['vqe'][vqe_min_idx]
            
            summary = f"\n{'=' * 30} C₂H₂ RESULTS SUMMARY {'=' * 30}\n"
            summary += f"Basis set: {BASIS}\n"
            summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_SPATIAL_ORBITALS}o\n"
            summary += f"Equilibrium {distance_type} distance: {eq_distance:.3f} Å\n"
            summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
            
            # Compare with reference values
            if not np.isnan(self.results['rhf'][vqe_min_idx]):
                rhf_energy = self.results['rhf'][vqe_min_idx]
                summary += f"RHF energy: {rhf_energy:.8f} Hartree\n"
                summary += f"Correlation energy: {min_vqe - rhf_energy:.8f} Hartree\n"
            
            if 'mp2' in self.results and not np.isnan(self.results['mp2'][vqe_min_idx]):
                mp2_energy = self.results['mp2'][vqe_min_idx]
                summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
                summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
            
            if not np.isnan(self.results['casscf'][vqe_min_idx]):
                casscf_energy = self.results['casscf'][vqe_min_idx]
                summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
                summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
            
            summary += f"{'=' * 80}\n"
            return summary
        except Exception as e:
            return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"C₂H₂ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create simulator
    sim = AcetyleneSimulator(verbose=True)
    
    # Run C≡C bond scan
    results = sim.run_cc_scan()
    
    # Plot results
    sim.plot_results(save_path="c2h2_potential_curve.png")
    
    # Print summary
    print(sim.get_summary())
    
    # Optionally run C-H bond scan at equilibrium C≡C distance
    # min_cc_idx = np.nanargmin(sim.results['vqe'])
    # eq_cc_distance = sim.results['cc_distances'][min_cc_idx]
    # ch_results = sim.run_ch_scan(cc_distance=eq_cc_distance)

In [ ]:
"C₂H₄"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from qiskit.primitives import Estimator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, L_BFGS_B

from pyscf import gto, scf, mcscf, mp

# --- Best settings for C₂H₄ ---
BASIS = "6-31g"  # Good compromise for ethylene
# Active space: focus on double bond and C-H bonds
ACTIVE_ELECTRONS = (6, 6)  # 12 valence electrons (6 alpha, 6 beta)
ACTIVE_SPATIAL_ORBITALS = 12  # 12 spatial orbitals for valence space
# Skip C 1s core orbitals (first 2 orbitals for each C)
ACTIVE_ORBITALS = list(range(4, 16))  # Skip 4 core orbitals
MAX_ITER = 400  # Good number of iterations for ethylene

class EthyleneSimulator:
    """Class for simulating ethylene (C₂H₄) using VQE with best practices."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("Ethylene (C₂H₄) VQE Simulation with Best Practices")
            print(f"{'='*50}")
            print(f"Basis: {BASIS}")
            print(f"Active Space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_SPATIAL_ORBITALS}o")
            print(f"{'='*50}\n")
    
    def get_c2h4_geometry(self, cc_distance=1.33):
        """
        Generate planar ethylene geometry.
        
        Args:
            cc_distance: C=C bond length in Å (default: 1.33)
            
        Returns:
            List of tuples with atom type and coordinates
        """
        # Standard bond parameters
        ch_distance = 1.09  # Å
        h_angle = 120.0  # degrees
        
        # Convert to radians
        h_angle_rad = h_angle * np.pi / 180.0
        
        # Calculate coordinates
        x_h = ch_distance * np.sin(h_angle_rad)
        y_h = ch_distance * np.cos(h_angle_rad)
        
        return [
            # Carbon atoms along z-axis
            ("C", (0.0, 0.0, -cc_distance/2)),
            ("C", (0.0, 0.0, cc_distance/2)),
            # Hydrogen atoms in the xz plane
            ("H", (x_h, 0.0, -cc_distance/2)),
            ("H", (-x_h, 0.0, -cc_distance/2)),
            ("H", (x_h, 0.0, cc_distance/2)),
            ("H", (-x_h, 0.0, cc_distance/2)),
        ]
    
    def format_geometry(self, geometry):
        """Convert geometry to string format."""
        atom_str = ""
        for atom in geometry:
            atom_str += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_str.strip("; ")
    
    def build_pyscf_mol(self, geometry):
        """Create PySCF molecule object."""
        atom_str = self.format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_str
        mol.basis = BASIS
        mol.charge = 0
        mol.spin = 0
        mol.build()
        return mol
    
    def run_reference_calcs(self, mol):
        """Run reference classical calculations."""
        results = {}
        
        # Run RHF
        try:
            mf = scf.RHF(mol)
            results["rhf"] = mf.kernel()
            mf_obj = mf
        except Exception as e:
            print(f"RHF calculation failed: {e}")
            results["rhf"] = np.nan
            mf_obj = None
        
        # Run MP2 for dynamic correlation
        try:
            mp2 = mp.MP2(mf_obj)
            mp2.kernel()
            # MP2 correlation energy + RHF energy
            results["mp2"] = mp2.e_corr + results["rhf"]
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = np.nan
        
        # Run CASSCF with same active space as VQE
        try:
            mf_cas = scf.RHF(mol).run()
            mycas = mcscf.CASSCF(mf_cas, 
                                ncas=ACTIVE_SPATIAL_ORBITALS, 
                                nelecas=sum(ACTIVE_ELECTRONS))
            results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF calculation failed: {e}")
            results["casscf"] = np.nan
        
        return results
    
    def build_qiskit_problem(self, geometry):
        """Build Qiskit Nature problem with proper active space."""
        atom_str = self.format_geometry(geometry)
        
        # Create driver and run
        driver = PySCFDriver(
            atom=atom_str, 
            basis=BASIS, 
            unit=DistanceUnit.ANGSTROM, 
            charge=0, 
            spin=0
        )
        problem = driver.run()
        
        # Print full system information
        if self.verbose:
            print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
                  f"{problem.num_particles} electrons")
        
        # Apply active space transformation with explicit orbital selection
        transformer = ActiveSpaceTransformer(
            num_electrons=ACTIVE_ELECTRONS,
            num_spatial_orbitals=ACTIVE_SPATIAL_ORBITALS,
            active_orbitals=ACTIVE_ORBITALS  # Skip core orbitals
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Active space: {sum(problem.num_particles)} electrons in "
                  f"{problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe(self, problem):
        """Run VQE with optimal settings."""
        # Use parity mapper with symmetry reduction
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create initial state and ansatz
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        if self.verbose:
            print(f"UCCSD parameters: {ansatz.num_parameters}")
        
        # Better initialization - small random values with seed for reproducibility
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Use SLSQP optimizer - works well for ethylene
        optimizer = SLSQP(maxiter=MAX_ITER)
        
        # Create CPU estimator
        estimator = Estimator()
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE and time it
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get interpreted energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.1f} seconds")
            print(f"VQE energy: {energy:.8f} Hartree")
            print(f"Optimizer evaluations: {result.optimizer_evals}")
        
        return energy, result
    
    def run_cc_scan(self, cc_distances=None):
        """
        Run energy scan over C=C bond distances.
        
        Args:
            cc_distances: List of C=C distances to scan (default: around equilibrium)
        """
        if cc_distances is None:
            # Default distances: focused around equilibrium (~1.33 Å)
            cc_distances = np.linspace(1.1, 1.6, 9)
        
        vqe_energies = []
        rhf_energies = []
        mp2_energies = []
        casscf_energies = []
        
        for i, cc_d in enumerate(cc_distances):
            print(f"\n{'-'*30}")
            print(f"C=C Distance: {cc_d:.3f} Å ({i+1}/{len(cc_distances)})")
            print(f"{'-'*30}")
            
            # Generate geometry
            geometry = self.get_c2h4_geometry(cc_distance=cc_d)
            
            # Run classical references
            mol = self.build_pyscf_mol(geometry)
            ref_results = self.run_reference_calcs(mol)
            rhf_energies.append(ref_results.get("rhf"))
            mp2_energies.append(ref_results.get("mp2"))
            casscf_energies.append(ref_results.get("casscf"))
            
            # Run VQE
            try:
                problem = self.build_qiskit_problem(geometry)
                vqe_energy, _ = self.run_vqe(problem)
                vqe_energies.append(vqe_energy)
                
                # Print comparison
                print("\nEnergy comparison at C=C distance = {:.3f} Å:".format(cc_d))
                print(f"  VQE:    {vqe_energy:.8f} Hartree")
                print(f"  RHF:    {ref_results.get('rhf', 'N/A')}")
                if ref_results.get("mp2") is not None:
                    print(f"  MP2:    {ref_results.get('mp2'):.8f} Hartree")
                if ref_results.get("casscf") is not None:
                    print(f"  CASSCF: {ref_results.get('casscf'):.8f} Hartree")
            
            except Exception as e:
                print(f"VQE calculation failed: {e}")
                vqe_energies.append(np.nan)
        
        # Store results
        self.results = {
            'cc_distances': cc_distances,
            'vqe': vqe_energies,
            'rhf': rhf_energies,
            'mp2': mp2_energies,
            'casscf': casscf_energies
        }
        
        return self.results
    
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot. Run a scan first.")
            return
        
        plt.figure(figsize=(10, 6))
        
        distances = self.results['cc_distances']
        
        # Plot VQE energies
        plt.plot(
            distances,
            self.results['vqe'],
            'o-',
            label='VQE (active space)',
            color='limegreen',
            linewidth=2,
            markersize=8
        )
        
        # Plot RHF energies
        if any(~np.isnan(self.results['rhf'])):
            plt.plot(
                distances,
                self.results['rhf'],
                's--',
                label='RHF',
                color='red',
                linewidth=2,
                markersize=6
            )
        
        # Plot MP2 energies if available
        if 'mp2' in self.results and any(~np.isnan(self.results['mp2'])):
            plt.plot(
                distances,
                self.results['mp2'],
                'x-',
                label='MP2',
                color='purple',
                linewidth=2,
                markersize=6
            )
        
        # Plot CASSCF energies
        if any(~np.isnan(self.results['casscf'])):
            plt.plot(
                distances,
                self.results['casscf'],
                'd-',
                label='CASSCF',
                color='blue',
                linewidth=2,
                markersize=6
            )
        
        # Plot styling
        plt.xlabel("C=C Distance (Å)", fontsize=14)
        plt.ylabel("Energy (Hartree)", fontsize=14)
        plt.title(f"C₂H₄ Potential Energy Curve ({BASIS} basis)", fontsize=16)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        
        # Add text box with equilibrium information
        try:
            vqe_min_idx = np.nanargmin(self.results['vqe'])
            eq_distance = distances[vqe_min_idx]
            min_energy = self.results['vqe'][vqe_min_idx]
            
            textstr = f"Equilibrium C=C distance: {eq_distance:.3f} Å\n"
            textstr += f"VQE minimum energy: {min_energy:.6f} Ha"
            
            props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
            plt.text(0.05, 0.05, textstr, transform=plt.gca().transAxes, 
                     fontsize=12, verticalalignment='bottom', bbox=props)
        except:
            pass
        
        if save_path:
            plt.savefig(save_path, dpi=300)
            print(f"Plot saved to {save_path}")
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize."
        
        try:
            # Find minimum energies
            vqe_min_idx = np.nanargmin(self.results['vqe'])
            distances = self.results['cc_distances']
            eq_distance = distances[vqe_min_idx]
            min_vqe = self.results['vqe'][vqe_min_idx]
            
            summary = f"\n{'=' * 30} C₂H₄ RESULTS SUMMARY {'=' * 30}\n"
            summary += f"Basis set: {BASIS}\n"
            summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_SPATIAL_ORBITALS}o\n"
            summary += f"Equilibrium C=C distance: {eq_distance:.3f} Å (exp. ~1.33 Å)\n"
            summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
            
            # Compare with reference values
            if not np.isnan(self.results['rhf'][vqe_min_idx]):
                rhf_energy = self.results['rhf'][vqe_min_idx]
                summary += f"RHF energy: {rhf_energy:.8f} Hartree\n"
                summary += f"Correlation energy: {min_vqe - rhf_energy:.8f} Hartree\n"
            
            if 'mp2' in self.results and not np.isnan(self.results['mp2'][vqe_min_idx]):
                mp2_energy = self.results['mp2'][vqe_min_idx]
                summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
                summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
            
            if not np.isnan(self.results['casscf'][vqe_min_idx]):
                casscf_energy = self.results['casscf'][vqe_min_idx]
                summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
                summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
            
            summary += f"{'=' * 80}\n"
            return summary
        except Exception as e:
            return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"C₂H₄ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Create simulator
    sim = EthyleneSimulator(verbose=True)
    
    # Run C=C bond scan
    results = sim.run_cc_scan()
    
    # Plot results
    sim.plot_results(save_path="c2h4_potential_curve.png")
    
    # Print summary
    print(sim.get_summary())